## Data cleaning of pharmacy sales indications

In [2]:
import os
import glob

# Data handling and manipulation library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# Set print options
pd.set_option('display.max_columns', None)

### some stuff for making pretty plots
from matplotlib import rcParams
from cycler import cycler
## Set plotting style and print options
sns.set_theme()
sns.set_style("whitegrid")
sns.set_context("paper") #Possible are paper, notebook, talk and poster
# Set figure format
wide_format, square_format = False, True
if wide_format:
    d = {'lines.linewidth': 2, 'axes.titlesize': 18, 'axes.labelsize': 18, 'xtick.labelsize': 12, 'ytick.labelsize': 12,\
     'legend.fontsize': 15, 'font.family': 'serif', 'figure.figsize': (9,6)}
if square_format:
    d = {'lines.linewidth': 2, 'axes.titlesize': 16, 'axes.labelsize': 14, 'xtick.labelsize': 10, 'ytick.labelsize': 10,\
     'legend.fontsize': 12, 'font.family': 'serif', 'figure.figsize': (6,6)}
    
d_colors = {'axes.prop_cycle': cycler(color = ['teal', 'navy', 'coral', 'plum', 'purple', 'olivedrab',\
         'black', 'red', 'cyan', 'yellow', 'khaki','lightblue'])}
rcParams.update(d)
rcParams.update(d_colors)

Get an overview of all the files in the indication insights folder

In [12]:
# Set the path to the data directory
data_dir = "../../../Indication Insights Data-20240905"

# List all files in the data directory
files = glob.glob(os.path.join(data_dir, '*.csv'))
file_names = os.listdir(data_dir)
file_names = [file.lower() for file in file_names if file.endswith('.csv')]

# Load all data files into a dictionary
df_dict = {}
for file in files:
    df = pd.read_csv(file, sep=';',encoding='iso-8859-1')
    df_dict[os.path.basename(file).lower().split('.csv')[0]] = df

keys = list(df_dict.keys())
file_names

['diagnosis_groups.csv',
 'diagnosis_groups_sks diagnoser.csv',
 'diagnosis_groups_spa.csv',
 'dosing.csv',
 'hospital_account.csv',
 'hospital_region.csv',
 'salgs_data_indication_insight_hospital.csv',
 'salgs_data_indication_insight_pharmacy.csv',
 'substancesofinterest.csv',
 'tabel_sks_diagnose_country.csv',
 'tabel_sks_diagnose_hospital.csv',
 'tabel_sks_diagnose_region.csv']

Clean 'salgs_data_indication_insight_pharmacy'

In [13]:
choose_file = 'salgs_data_indication_insight_pharmacy'
sales_ins = df_dict[choose_file]
sales_ins.head()

,ï»¿Year Month,Region,Account Description,Product,Size,Size Numeric,Strength,WHO ATC 5 Code,Volume,Value,Unnamed: 10
0,201003.0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,1.0,"8083,55",NaN
1,201006.0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,1.0,"8083,55",NaN
2,201007.0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,1.0,"8083,55",NaN
3,201011.0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,2.0,"16167,1",NaN
4,201103.0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,1.0,"8406,89",NaN


In [14]:
# choose columns to drop
cols_to_drop = ['Unnamed: 10']
# drop columns from sales_ins if they exist
for col in cols_to_drop:
    if col in sales_ins.columns:
        sales_ins.drop(col, axis=1, inplace=True)

In [15]:
# check for missing values in the data per column
missing_values = sales_ins.isnull().sum()
missing_values

ï»¿Year Month          47604
Region                 47604
Account Description    47604
Product                47604
Size                   47604
Size Numeric           47604
Strength               47604
WHO ATC 5 Code         47604
Volume                 47604
Value                  47604
dtype: int64

In [16]:
# drop rows with missing values
sales_ins.dropna(inplace=True)

In [17]:
cols_to_format = ['Region', 'Account Description']

# print unique values and their frequency for each column
for col in cols_to_format:
    print(sales_ins[col].value_counts())

Region
Syddanmark         2337
SjÃ¦lland          1608
Midtjylland        1375
Nordjylland         982
Hovedstaden         781
FÃ¦rÃ¸erne          286
Danmark diverse      10
Name: count, dtype: int64
Account Description
4060 Centralapoteket, Region Hovedstaden          653
4100 Ãrhus Kommunehospitalet, apoteket           602
4070 SjÃ¦llands Universitetshospital, Roskilde    572
4120 Odense Universitetshospital                  570
4130 Aalborg Sygehus, apoteket                    411
                                                 ... 
2491 Ikast Apotek Vest                              1
2620 Gammel Kongevej apotek                         1
3681 Ãlby Apotek                                   1
3611 Aalborg Storcenter Apotek                      1
3680 KÃ¸ge Boholte Apotek                           1
Name: count, Length: 162, dtype: int64


In [18]:
# rename 'ï»¿Year Month' column to 'Year Month'
sales_ins.rename(columns={'ï»¿Year Month': 'Year Month'}, inplace=True)

# replace the weird characters in the columns with æ, ø, å
for col in cols_to_format:
    sales_ins[col] = sales_ins[col].apply(lambda x: x.replace('Ã¸','ø'))
    sales_ins[col] = sales_ins[col].apply(lambda x: x.replace('Ã\x98','ø'))
    sales_ins[col] = sales_ins[col].apply(lambda x: x.replace('Ã¦','æ'))
    sales_ins[col] = sales_ins[col].apply(lambda x: x.replace('Ã\x86','æ'))
    sales_ins[col] = sales_ins[col].apply(lambda x: x.replace('Ã¥','å'))
    sales_ins[col] = sales_ins[col].apply(lambda x: x.replace('Ã','aa'))

sales_ins.head(8)

,Year Month,Region,Account Description,Product,Size,Size Numeric,Strength,WHO ATC 5 Code,Volume,Value
0,201003.0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,1.0,"8083,55"
1,201006.0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,1.0,"8083,55"
2,201007.0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,1.0,"8083,55"
3,201011.0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,2.0,"16167,1"
4,201103.0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,1.0,"8406,89"
5,201103.0,Danmark diverse,0009 GROSSISTSALG,Stelara,"1 stk. (0,5 ml)",1.0,45 mg,L04AC05,5.0,"138080,55"
6,201105.0,Danmark diverse,0009 GROSSISTSALG,Remicade,1 htgl. a 100 mg,100.0,100 mg,L04AB02,1.0,"4406,8"
7,201108.0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,2.0,"16167,1"


Now we convert the year month column (yyyymm.0) into (yymm.0) after 2000.

In [20]:
# only do this if the column exists
if 'Year Month' in sales_ins.columns:   
    # Convert 'Year Month' column to Year Month after the year 2000
    sales_ins['Year Month'] = sales_ins['Year Month'].astype(str)
    sales_ins['Year Month (after 2000)'] = sales_ins['Year Month'].str[2:]

    # Add month and year columns to the sales data
    # We start by making the column into a string to make it easier to slice
    sales_ins['Year Month (after 2000)'] = sales_ins['Year Month'].str[2:]#  .astype(str)

    sales_ins.drop('Year Month', axis=1, inplace=True)
    
sales_ins.head(3)

,Region,Account Description,Product,Size,Size Numeric,Strength,WHO ATC 5 Code,Volume,Value,Year Month (after 2000)
0,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,1.0,"8083,55",1003.0
1,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,1.0,"8083,55",1006.0
2,Danmark diverse,0009 GROSSISTSALG,Cimzia,2 stk.,2.0,200 mg,L04AB05,1.0,"8083,55",1007.0


Let's have a look at the values and their frequency across columns

In [46]:
sales_ins[sales_ins['Year Month (after 2000)'].str.split('.').str[0].astype(int) >= 2000].sort_values('Year Month (after 2000)', ascending=False)

,Region,Account Description,Product,Size,Size Numeric,Strength,WHO ATC 5 Code,Volume,Value,Year Month (after 2000)
3239,Hovedstaden,"4060 Centralapoteket, Region Hovedstaden",Remicade,1 htgl.,1.0,100 mg,L04AB02,90.0,332199,2006.0
159,Midtjylland,0060 aarhus Løve apotek,Stelara,"1 stk. (0,5 ml)",1.0,45 mg,L04AC05,1.0,"22677,47",2006.0
358,Sjælland,0140 Køge Torvets apotek,Stelara,1 eng. spr. a 1 ml,1.0,90 mg,L04AC05,1.0,"22677,47",2006.0
613,Syddanmark,0511 Varde østervold Apotek,Stelara,"1 stk. (0,5 ml)",1.0,45 mg,L04AC05,1.0,"22677,47",2006.0
5102,Syddanmark,4120 Odense Universitetshospital,Zessly,3 stk.,3.0,100 mg,L04AB02,580.0,6408420,2006.0
...,...,...,...,...,...,...,...,...,...,...
5478,Nordjylland,"4130 Aalborg Sygehus, apoteket",Remicade,1 htgl.,1.0,100 mg,L04AB02,16.0,"59057,6",2001.0
5479,Nordjylland,"4130 Aalborg Sygehus, apoteket",Stelara,1 eng. spr. a 1 ml,1.0,90 mg,L04AC05,2.0,"45354,94",2001.0
5482,Nordjylland,"4130 Aalborg Sygehus, apoteket",Zessly,3 stk.,3.0,100 mg,L04AB02,-2.0,-22098,2001.0
934,Sjælland,1000 Haslev apotek,Stelara,1 eng. spr. a 1 ml,1.0,90 mg,L04AC05,1.0,"22677,47",2001.0


In [25]:
for col in sales_ins.columns:
    #print(f'{col}: {sales_ins[col].nunique()}')
    print(sales_ins[col].value_counts())
    print("\n")

Region
Syddanmark         2337
Sjælland           1608
Midtjylland        1375
Nordjylland         982
Hovedstaden         781
Færøerne            286
Danmark diverse      10
Name: count, dtype: int64


Account Description
4060 Centralapoteket, Region Hovedstaden         653
4100 aarhus Kommunehospitalet, apoteket          602
4070 Sjællands Universitetshospital, Roskilde    572
4120 Odense Universitetshospital                 570
4130 Aalborg Sygehus, apoteket                   411
                                                ... 
2491 Ikast Apotek Vest                             1
2620 Gammel Kongevej apotek                        1
3681 ølby Apotek                                   1
3611 Aalborg Storcenter Apotek                     1
3680 Køge Boholte Apotek                           1
Name: count, Length: 162, dtype: int64


Product
Stelara      3776
Remicade     1480
Cimzia       1341
Remsima       412
Inflectra     197
Zessly        173
Name: count, dtype: int64


Size
1 st

Now we remove the Danmark diverse since its occurence is rare.

In [26]:
# Drop 'Danmark diverse' from the Region column (if not already done)
try:
    drop_mask = sales_ins['Region'].str.contains('Danmark diverse')
    sales_ins = sales_ins[~drop_mask]
except:
    # do nothing if the column has already been dropped
    pass
sales_ins['Region'].value_counts()    

Region
Syddanmark     2337
Sjælland       1608
Midtjylland    1375
Nordjylland     982
Hovedstaden     781
Færøerne        286
Name: count, dtype: int64

In [30]:
sales_ins['Strength']

10       90 mg
11       90 mg
12       90 mg
13       90 mg
14       90 mg
         ...  
7374    100 mg
7375    200 mg
7376    100 mg
7377    200 mg
7378    100 mg
Name: Strength, Length: 7369, dtype: object